In [31]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset,  DataLoader
import torch
from pytorch_transformers import *
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
from pytorch_pretrained_bert.optimization import BertAdam, WarmupLinearSchedule
print(device)
import math
from scipy import spatial
from torch.autograd import Variable
import torch.nn.functional as F



cuda


In [2]:
import csv
SemEval_prefix = "SemEval_"
import pandas as pd
import numpy as np

class Parse_data:

    def  __init__ (self, file_path, testing_emotions_path):
        self.file_path = file_path
        self.testing_emotions_path = testing_emotions_path
        self.sem_eval2007_id_to_emotions = self.get_sem_eval2007_id_to_emotions()
        self.data_from_file = pd.read_csv(self.file_path) 
        self.words_to_ix = self.get_words_to_ix()
        self.normalize_data()
        self.train_data()

    def get_words_to_ix(self):
        results = set()
        dict = {}
        self.data_from_file['text'].str.split().apply(results.update)
        for i, word in enumerate(results):
            dict.update({word : i})
        return dict
        
        
    def train_data(self):
        
        boolean_indexes = self.data_from_file["id"].str.startswith(SemEval_prefix)
        self.train_instances = self.data_from_file[~boolean_indexes]
        self.test_instances = self.data_from_file[boolean_indexes]

        extra_column = [np.nan] * len(self.test_instances)
        self.test_instances.insert(5, "Categorial", extra_column, True)

        for index in self.sem_eval2007_id_to_emotions:
            value = self.sem_eval2007_id_to_emotions[index]
            self.test_instances.loc[self.test_instances.id == SemEval_prefix + str(index), "Categorial"] = [value]

        self.test_instances = self.test_instances.dropna(axis=0, how='any')

        train_instances = list(range(len(self.train_instances)))
        self.train_instances.reindex(train_instances)
        test_instances = list(range(len(self.test_instances)))
        self.test_instances.reindex(test_instances)


    def normalize_data(self):

        #Train
        #mean = self.train_instances.mean()
        #std = self.train_instances.std()
        maxi = self.data_from_file.max()
        mini = self.data_from_file.min()

        current_v = self.data_from_file["V"]
        normalized_v = 2*(current_v - mini["V"]) / (maxi["V"]-mini["V"]) - 1
        self.data_from_file["V"] = normalized_v

        current_a = self.data_from_file["A"]
        normalized_a =2* (current_a - mini["A"]) / (maxi["A"]-mini["A"]) -1 
        self.data_from_file["A"] = normalized_a
        
        current_d = self.data_from_file["D"]
        normalized_d = 2*(current_d - mini["D"]) / (maxi["D"]-mini["D"]) -1
        self.data_from_file["D"] = normalized_d


    def get_train_data(self):
        
        return self.train_instances


    def get_id_to_sentence_dict(self):
        sentences = {}
        with open(self.file_path, encoding='utf8') as csv_file:
            csv_reader = csv.reader(csv_file, delimiter=',')
            line_count = 0

            for row in csv_reader:
                if line_count == 0:
                    line_count += 1
                else:
                    if line_count == 393:
                        line_count += 1
                    sentences.update({row[0]: row[1]})
                    line_count += 1
        return sentences

    def get_sem_eval2007_id_to_emotions(self):
        sentences = {}
        with open(self.testing_emotions_path) as csv_file:
            csv_reader = csv.reader(csv_file, delimiter=' ')

            # row is in the form: 0: id  1: anger 2: disgust 3: fear 4: joy 5 :sadness 6: surprise
            for row in csv_reader:
                instance = {
                    "id": row[0],
                    "anger": row[1],
                    "disgust": row[2],
                    "fear": row[3],
                    "joy": row[4],
                    "sadness": row[5],
                    "surprise": row[6]
                }
                sentences.update({row[0]: instance})

        return sentences


    def get_test_data(self):
        return self.test_instances

# reader_path = "./corpus/reader.csv"
# writer_path = "./corpus/writer.csv"
# raw_path = "./corpus/raw.csv"
# sem2007_paths = "./affectivetext_test.emotions.gold"
#
# moss = parse_data(reader_path, writer_path, raw_path, sem2007_paths)
#
# a = moss.get_train_data()
# b = moss.get_test_data()
# a = 3

/anaconda/envs/py35/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/anaconda/envs/py35/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


In [11]:
file_path = "/home/ofer/emobank.csv"
emotions_path = "/home/ofer/affectivetext_test.emotions.csv"

In [12]:
parse_data_instance = Parse_data(file_path, emotions_path)

/anaconda/envs/py35/lib/python3.5/site-packages/pandas/core/indexing.py:543: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


In [13]:
class EmoBankDatabase(Dataset):
    """Face Landmarks dataset."""

    def __init__(self, file_path, emotions_path, is_testing = False):
        self.index_to_tensors = {}
        parse_data_instance = Parse_data(file_path, emotions_path)
        if is_testing:
                self.par_data = parse_data_instance.get_test_data()
        else:
            self.par_data = parse_data_instance.get_train_data()

    def __len__(self):
        return len(self.par_data)

    def __getitem__(self, idx):
        item = self.par_data.iloc[idx]
        try:
            tens = self.index_to_tensors[idx]
        except:
            tens = torch.tensor([item.V, item.A, item.D])
            self.index_to_tensors.update({idx: tens})
        return item.text, tens

In [14]:
class EmobankRnnModel(torch.nn.Module):
    def __init__(self):
        """
        In the constructor we instantiate two nn.Linear modules and assign them as
        member variables.
        """
        super(EmobankRnnModel, self).__init__()

        #         self.embeddings = nn.Embedding(len(words_to_index), 20)
        self.lstm = nn.RNN(20, 100, dropout=0.05)
        self.linear2 = torch.nn.Linear(100, 100)
        self.linear3 = torch.nn.Linear(100, 75)
        self.linear4 = torch.nn.Linear(75, 75)
        self.linear5 = torch.nn.Linear(75, 3)

    def forward(self, x):
        """
        In the forward function we accept a Tensor of input data and we must return
        a Tensor of output data. We can use Modules defined in the constructor as
        well as arbitrary operators on Tensors.
        """
        hidden_layers = []

        for sentence in x:
            self.hidden = self.init_hidden()
            lstm_out, self.hidden = self.lstm(sentence.view(len(sentence), 1, -1), self.hidden)
            hidden_layers.append(self.hidden[0].reshape(-1))
        x = torch.stack(hidden_layers)
        
        x = F.relu(self.linear2(x))
        x = F.relu(self.linear3(x))
        x = F.relu(self.linear4(x))
        x = self.linear5(x)

        return x

    def init_hidden(self):
        # the weights are of the form (nb_layers, batch_size, nb_lstm_units)
        hidden_a = torch.zeros(1, 1, 100).to(device)
        hidden_b = torch.randn(1, 1, 100).to(device)
        return hidden_a    
        
        hidden_a = Variable(hidden_a)
        hidden_b = Variable(hidden_b)




In [15]:
class EmobankGRUModel(torch.nn.Module):
    def __init__(self):
        """
        In the constructor we instantiate two nn.Linear modules and assign them as
        member variables.
        """
        super(EmobankGRUModel, self).__init__()

        #         self.embeddings = nn.Embedding(len(words_to_index), 20)
        self.lstm = nn.GRU(20, 100, dropout=0.05)
        self.linear2 = torch.nn.Linear(100, 100)
        self.linear3 = torch.nn.Linear(100, 75)
        self.linear4 = torch.nn.Linear(75, 75)
        self.linear5 = torch.nn.Linear(75, 3)

    def forward(self, x):
        """
        In the forward function we accept a Tensor of input data and we must return
        a Tensor of output data. We can use Modules defined in the constructor as
        well as arbitrary operators on Tensors.
        """
        hidden_layers = []

        for sentence in x:
            self.hidden = self.init_hidden()
            lstm_out, self.hidden = self.lstm(sentence.view(len(sentence), 1, -1), self.hidden)
            hidden_layers.append(self.hidden[0].reshape(-1))
        x = torch.stack(hidden_layers)
        
        x = F.relu(self.linear2(x))
        x = F.relu(self.linear3(x))
        x = F.relu(self.linear4(x))
        x = self.linear5(x)

        return x

    def init_hidden(self):
        # the weights are of the form (nb_layers, batch_size, nb_lstm_units)
        hidden_a = torch.zeros(1, 1, 100).to(device)
        hidden_b = torch.randn(1, 1, 100).to(device)
        return hidden_a    
        
        hidden_a = Variable(hidden_a)
        hidden_b = Variable(hidden_b)




In [16]:
class EmobankLSTMModel(torch.nn.Module):
    def __init__(self):
        """
        In the constructor we instantiate two nn.Linear modules and assign them as
        member variables.
        """
        super(EmobankLSTMModel, self).__init__()

        #         self.embeddings = nn.Embedding(len(words_to_index), 20)
        self.lstm = nn.LSTM(20, 100, dropout=0.05)
        self.linear2 = torch.nn.Linear(100, 100)
        self.linear3 = torch.nn.Linear(100, 75)
        self.linear4 = torch.nn.Linear(75, 75)
        self.linear5 = torch.nn.Linear(75, 3)

    def forward(self, x):
        """
        In the forward function we accept a Tensor of input data and we must return
        a Tensor of output data. We can use Modules defined in the constructor as
        well as arbitrary operators on Tensors.
        """
        hidden_layers = []

        for sentence in x:
            self.hidden = self.init_hidden()
            lstm_out, self.hidden = self.lstm(sentence.view(len(sentence), 1, -1), self.hidden)
            hidden_layers.append(self.hidden[0].reshape(-1))
        x = torch.stack(hidden_layers)
        
        x = F.relu(self.linear2(x))
        x = F.relu(self.linear3(x))
        x = F.relu(self.linear4(x))
        x = self.linear5(x)

        return x

    def init_hidden(self):
        # the weights are of the form (nb_layers, batch_size, nb_lstm_units)
        hidden_a = torch.zeros(1, 1, 100).to(device)
        hidden_b = torch.randn(1, 1, 100).to(device)
        
        return (hidden_a, hidden_b)




In [17]:
class EmobankFineTuningBERTModule(torch.nn.Module):
    def __init__(self, model_class):
        """
        In the constructor we instantiate two nn.Linear modules and assign them as
        member variables.
        """
        super(EmobankFineTuningBERTModule, self).__init__()

        self.bert = model_class
        self.linear2 = torch.nn.Linear(768, 100)
        self.linear3 = torch.nn.Linear(100, 75)
        self.linear4 = torch.nn.Linear(75, 75)
        self.linear5 = torch.nn.Linear(75, 3)

    def forward(self, x):
        """
        In the forward function we accept a Tensor of input data and we must return
        a Tensor of output data. We can use Modules defined in the constructor as
        well as arbitrary operators on Tensors.
        """
        x = [get_all_hidden_states(instance, self.bert) for instance in x]
        x = [self.linear2(instance.view(-1, 768)) for instance in x]
        x = [torch.mean(instance, dim=0) for instance in x]
        x = torch.stack(x, dim=0, out=None)
        
        x = F.relu(self.linear3(x))
        x = F.relu(self.linear4(x))
        x = self.linear5(x)

        
        
        return x

In [56]:
def get_all_hidden_states(x, model):
    all_hidden_states, all_attentions = model(x)[-2:]

    return all_hidden_states[0]


def get_all_hidden_states(x, model):
    all_hidden_states, all_attentions = model(x)[-2:]

    return all_hidden_states[0]

def prepare_sequence(seq, to_ix, embeddings):
    seq = seq.split()
    idxs = [to_ix[w] for w in seq]
    return embeddings(torch.tensor(idxs, dtype=torch.long))



def Train_Net_sequencial(my_net, trainloader, criterion):
    optimizer = torch.optim.SGD(my_net.parameters(), lr=0.01, momentum=0.9)

    embeddings = nn.Embedding(len(parse_data_instance.words_to_ix), 20)

    # training loop
    for epoch in range(1000):
        if epoch > 0:
            #path = "C:\\IDCMaster\\DeepLearning\\projectNotebook\\modelCosinev7" + str(epoch + 83)
            #path2 = "C:\\IDCMaster\\DeepLearning\\projectNotebook\\tokinezerv" + str(epoch + 83)
            #torch.save(my_net, path)
            print("Avg epoch loss for training data: " + str(epoch_loss / len(trainloader)))
        epoch_loss = 0.0
        running_loss = 0.0
        for i, data in enumerate(trainloader, 0):
            # get the inputs

            inputs, labels = data

            inputs = [prepare_sequence(sentence, parse_data_instance.words_to_ix, embeddings) for sentence in inputs]

            labels = labels.to(device)
            #             embedded_input = [torch.tensor([tokenizer.encode(text)]).to(device) for text in inputs]

            # flags = Variable(torch.ones(len(embedded_input))).to(device)

            # zero the parameter gradients
            optimizer.zero_grad()

            # forward + backward + optimize
            outputs = my_net(inputs)
            # loss = torch.mean(torch.sqrt(torch.sum((outputs - labels) ** 2,  dim=1))).to(device)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            # print statistics
            running_loss += loss.item()
            epoch_loss += loss.item()

        # After every epoch, measure the testing
        sum_loss = 0.0
        for i, data in enumerate(dataloader_testing, 0):
            # get the inputs
            inputs, labels = data
            labels = labels.to(device)
            # embedded_input = [torch.tensor([tokenizer.encode(text)]).to(device) for text in inputs]
            my_net.eval()
            inputs = [prepare_sequence(sentence, parse_data_instance.words_to_ix, embeddings) for sentence in inputs]
            outputs = my_net(inputs)
            # flags = Variable(torch.ones(len(embedded_input))).to(device)
            loss = criterion(outputs, labels)
            sum_loss = sum_loss + loss.item()
        print("finished epoch " + str(epoch) + " with avarage sum_loss on testing data = " + str(sum_loss / i))

    return my_net


def Train_Net_BERT(my_net, trainloader, criterion):
   #optimizer = torch.optim.SGD(my_net.parameters(), lr=0.01, momentum=0.9)
    optimizer = BertAdam(my_net.parameters(), lr=0.001, schedule='warmup_linear', warmup=0.1, t_total=23000000)
    print(device)

    # training loop
    for epoch in range(1000):
        if epoch > 0:
            #path = "/home/ofer/model" + str(epoch+83)
            #path2 = "/home/ofer/tokinezer" + str(epoch+83)
            #torch.save(my_net, path) 
            #torch.save(tokenizer, path2) 
            print("Avg epoch loss for training data: " + str(epoch_loss/len(trainloader)))
        epoch_loss = 0.0
        running_loss = 0.0
        for i, data in enumerate(trainloader, 0):
            if (i%1000 == 0):
                print(i)
            # get the inputs
            inputs, labels = data
                
            labels = labels.to(device)
            embedded_input = [torch.tensor([tokenizer.encode(text)]).to(device) for text in inputs]
                
            #flags = Variable(torch.ones(len(embedded_input))).to(device)

            # forward + backward + optimize
            outputs = my_net(embedded_input)
            #loss = torch.mean(torch.sqrt(torch.sum((outputs - labels) ** 2,  dim=1))).to(device) 
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            # zero the parameter gradients
            optimizer.zero_grad()
            
            # print statistics
            running_loss += loss.item()
            epoch_loss += loss.item()
            
        # After every epoch, measure the testing
        sum_loss = 0.0
        for i, data in enumerate(dataloader_testing, 0):
            # get the inputs
            inputs, labels = data
            labels = labels.to(device)
            embedded_input = [torch.tensor([tokenizer.encode(text)]).to(device) for text in inputs]
            my_net.eval()
            #inputs = [prepare_sequence(sentence, parse_data_instance.words_to_ix, embeddings) for sentence in inputs]
            outputs = my_net(embedded_input)
            # flags = Variable(torch.ones(len(embedded_input))).to(device)
            loss = criterion(outputs, labels)
            sum_loss = sum_loss + loss.item()
        print("finished epoch " + str(epoch) + " with avarage sum_loss on testing data = " + str(sum_loss / i))

    return my_net



In [51]:



##########################################################################



########################################################################## Set  basic parameters

file_path = "/home/ofer/emobank.csv"
emotions_path = "/home/ofer/affectivetext_test.emotions.csv"



MODELS = [(BertModel,       BertTokenizer,      'bert-base-uncased'),
          (OpenAIGPTModel,  OpenAIGPTTokenizer, 'openai-gpt'),
          (GPT2Model,       GPT2Tokenizer,      'gpt2'),
          (TransfoXLModel,  TransfoXLTokenizer, 'transfo-xl-wt103'),
          (XLNetModel,      XLNetTokenizer,     'xlnet-base-cased'),
          (XLMModel,        XLMTokenizer,        'xlm-mlm-enfr-1024')]

model_class, tokenizer_class, pretrained_weights = MODELS[0]

In [52]:
emobank_dataset_training = EmoBankDatabase(file_path , emotions_path)
emobank_dataset_training
    

dataloader_training = DataLoader(emobank_dataset_training, batch_size=4,
                            shuffle=True, num_workers=6)

/anaconda/envs/py35/lib/python3.5/site-packages/pandas/core/indexing.py:543: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


In [53]:
emobank_dataset_testing = EmoBankDatabase(file_path , emotions_path, True)
dataloader_testing = DataLoader(emobank_dataset_testing, batch_size=1,
                        shuffle=True, num_workers=6)

number_of_instances = len(emobank_dataset_testing)

/anaconda/envs/py35/lib/python3.5/site-packages/pandas/core/indexing.py:543: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


In [44]:
parse_data_instance = Parse_data(file_path, emotions_path)

/anaconda/envs/py35/lib/python3.5/site-packages/pandas/core/indexing.py:543: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


In [45]:
len(parse_data_instance.words_to_ix)

28931

In [24]:
def prepare_sequence(seq, to_ix, embeddings):
    seq = seq.split()
    idxs = [to_ix[w] for w in seq]
    return embeddings(torch.tensor(idxs, dtype=torch.long))

In [46]:
test = parse_data_instance.get_test_data()

In [54]:
criterion = nn.MSELoss()
bert_model = BertModel.from_pretrained(pretrained_weights).to(device)
emobank_fine_tuning_bert_module = EmobankFineTuningBERTModule(bert_model).to(device)
tokenizer = tokenizer_class.from_pretrained(pretrained_weights)

In [ ]:
emobank_fine_tuning_bert_module = Train_Net_BERT(emobank_fine_tuning_bert_module, dataloader_training, criterion)






cuda
0
1000
2000
finished epoch 0 with avarage sum_loss on testing data = 0.042717748653496077
Avg epoch loss for training data: 0.032285347636545544
0
1000
2000
finished epoch 1 with avarage sum_loss on testing data = 0.03718980227517812
Avg epoch loss for training data: 0.026710756878873268
0
1000
2000
finished epoch 2 with avarage sum_loss on testing data = 0.03607948807056853
Avg epoch loss for training data: 0.02179707068333955
0
1000
2000
finished epoch 3 with avarage sum_loss on testing data = 0.03527036522615647
Avg epoch loss for training data: 0.01747157207981585
0
1000
2000
finished epoch 4 with avarage sum_loss on testing data = 0.03627000008590814
Avg epoch loss for training data: 0.013335979469252768
0
1000
2000
finished epoch 5 with avarage sum_loss on testing data = 0.037463997056793624
Avg epoch loss for training data: 0.009798168360190155
0
1000
2000
finished epoch 6 with avarage sum_loss on testing data = 0.03716644827416682
Avg epoch loss for training data: 0.007179

In [28]:
emobank_gru_module = EmobankGRUModel().to(device)

/anaconda/envs/py35/lib/python3.5/site-packages/torch/nn/modules/rnn.py:51: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.05 and num_layers=1
  "num_layers={}".format(dropout, num_layers))


In [29]:
emobank_gru_module = Train_Net_sequencial(emobank_gru_module, dataloader_training, criterion)


finished epoch 0 with avarage sum_loss on testing data = 0.04212493491660827
Avg epoch loss for training data: 0.03780677252632991
finished epoch 1 with avarage sum_loss on testing data = 0.0434301919551631
Avg epoch loss for training data: 0.03769670416972888
finished epoch 2 with avarage sum_loss on testing data = 0.04690990026586849
Avg epoch loss for training data: 0.03755955674505837
finished epoch 3 with avarage sum_loss on testing data = 0.044082009621375845
Avg epoch loss for training data: 0.03769106678714732
finished epoch 4 with avarage sum_loss on testing data = 0.04490167645585293
Avg epoch loss for training data: 0.03759821525216871
finished epoch 5 with avarage sum_loss on testing data = 0.044748333609565795
Avg epoch loss for training data: 0.03747993860095678
finished epoch 6 with avarage sum_loss on testing data = 0.044457777898067825
Avg epoch loss for training data: 0.037464205428048306
finished epoch 7 with avarage sum_loss on testing data = 0.041857097779699474
Av

KeyboardInterrupt: 

In [31]:
emobank_rnn_module = EmobankRnnModel().to(device)

/anaconda/envs/py35/lib/python3.5/site-packages/torch/nn/modules/rnn.py:51: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.05 and num_layers=1
  "num_layers={}".format(dropout, num_layers))


In [32]:
emobank_rnn_module = Train_Net_sequencial(emobank_rnn_module, dataloader_training, criterion)

finished epoch 0 with avarage sum_loss on testing data = 0.04753585378778019
Avg epoch loss for training data: 0.03788075814187003
finished epoch 1 with avarage sum_loss on testing data = 0.0488085781277116
Avg epoch loss for training data: 0.0376554570584926
finished epoch 2 with avarage sum_loss on testing data = 0.04425725227836615
Avg epoch loss for training data: 0.03749830334138092


KeyboardInterrupt: 

In [26]:
emobank_lstm_module = EmobankLSTMModel().to(device)

/anaconda/envs/py35/lib/python3.5/site-packages/torch/nn/modules/rnn.py:51: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.05 and num_layers=1
  "num_layers={}".format(dropout, num_layers))


In [30]:
emobank_lstm_module = Train_Net_sequencial(emobank_lstm_module, dataloader_training, criterion)

finished epoch 0 with avarage sum_loss on testing data = 0.04205203087481761
Avg epoch loss for training data: 0.03786249024449526
finished epoch 1 with avarage sum_loss on testing data = 0.045893294863892056
Avg epoch loss for training data: 0.03775530279211196
finished epoch 2 with avarage sum_loss on testing data = 0.044866237282444536
Avg epoch loss for training data: 0.037665797378338944
finished epoch 3 with avarage sum_loss on testing data = 0.04545731376329078
Avg epoch loss for training data: 0.03780656566021411
finished epoch 4 with avarage sum_loss on testing data = 0.04233708896079749
Avg epoch loss for training data: 0.03765356640064755


KeyboardInterrupt: 

In [23]:
def get_instance_category(test_instance):
    max_value_first = -1
    max_value_second = -1
    max_category_first = "moss"
    max_category_second = "moss"
    instance_categories = test_instance["Categorial"]
    for category in instance_categories:
        if float(instance_categories[category]) > max_value_second and category != "id":
            max_value_first = max_value_second
            max_value_second = float(instance_categories[category])
            max_category_first = max_category_second
            max_category_second = category
    return [max_category_first]
    

In [22]:
def get_distance(a, b):
    return (np.square(a - b)).mean(axis=None)
#math.sqrt((current_category_vad["v"] - model_output[0])**2 + (current_category_vad["a"] - model_output[1])**2 + (current_category_vad["d"] - model_output[2])**2)

In [21]:
def get_category(model_output):
    model_output = model_output[0]
    categories_distance = []
    for i, category in enumerate(category_to_index):
        # index: 0:V, 1:A, 2:D
        current_category_vad = categories_vad[category]
        a = np.empty(3)
        a[0] = current_category_vad["v"]
        a[1] = current_category_vad["a"]
        a[2] = current_category_vad["d"]
        distance_from_category = get_distance(a,model_output.to("cpu").detach().numpy())
        categories_distance.append(distance_from_category)
#     print(category_to_index)
#     print(categories_distance)
    return index_to_catogory[categories_distance.index(min(categories_distance))]
        

In [20]:
categories_vad = {
    "joy":{
        "v":0.76,
        "a":0.48,
        "d":0.35
    },
    "anger":{
        "v":-0.51,
        "a":0.59,
        "d":0.25
    },
      "sadness":{
        "v":-0.63,
        "a":-0.27,
        "d":-0.33
    },
      "fear":{
        "v":-0.64,
        "a":0.6,
        "d":-0.43
    },
      "disgust":{
        "v":-0.6,
        "a":0.35,
        "d":0.11
    },
    "surprise":{
        "v":0.4,
        "a":0.67,
        "d":-0.13
    }
}


index_to_catogory = {
    0 : "joy",
    1 : "anger",
    2 : "sadness",
    3 : "fear",
    4 : "disgust",
    5 : "surprise",
}

category_to_index = ["joy", "anger", "sadness", "fear", "disgust", "surprise"]


In [ ]:

total_count = 0
correct_count = 0
    
for i, test_insance in test.iterrows():
    text = test_insance["text"]
    embeded_text = torch.tensor([tokenizer.encode(text)]).to(device)
    outputs = model([embeded_text])


    category = get_category(outputs)
    instance_actual_category = get_instance_category(test_insance)

    if category in instance_actual_category:
        correct_count += 1 
    total_count += 1


In [ ]:
print("correctness: " + str(correct_count / total_count))